In [ ]:
import hashlib

result = hashlib.sha256(
    bytes("The quick brown fox jumps over the lazy dog",'utf-8'))

print(result.hexdigest())


In [ ]:
result = hashlib.sha256(
    b'The quick brown fox jumps over the lazy dog')


In [ ]:
result = hashlib.sha256(
    b'The quick brown fox jumps over the lazy dag')
print(result.hexdigest())

In [ ]:
pip install cryptography

In [ ]:
from cryptography.fernet import Fernet

# generate the shared key
shared_key = Fernet.generate_key()
print(shared_key)    

# base64 encoded, binary format
# A new key is generated each time you run this block of code
# e.g. b'ixXEfrz2NTJlxy1OhxXlsCiFf0Ycg_GL0Cy0MlgTv4U='

In [ ]:
# create an instance of the Fernet class
fernet = Fernet(shared_key)


In [ ]:
# encrypt the message with the shared key
ciphertext = fernet.encrypt(
    bytes("Secret message!",'utf-8'))  # remember to pass in a bytes 
                                       # array

In [ ]:
# write the encrypted message to file
with open('message.encrypted', 'wb') as f:
    f.write(ciphertext)


In [ ]:
# write the shared key to file
with open('symmetric_key.crypt', 'wb') as f:
    f.write(shared_key)


In [ ]:
with open('symmetric_key.crypt', 'rb') as f:
    shared_key = f.read()
print(shared_key)


In [ ]:
# create an instance of the Fernet class
fernet = Fernet(shared_key)# decrypt the encrypted message read from file
with open('message.encrypted', 'rb') as f:
    print(fernet.decrypt(f.read()).decode("utf-8"))


In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa

# generate the private key
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)

# derive the public key from the private key 
public_key = private_key.public_key()

In [ ]:
from cryptography.hazmat.primitives import serialization

#---serialize the private key as bytes---
pem = private_key.private_bytes(
    encoding = serialization.Encoding.PEM,
    format = serialization.PrivateFormat.PKCS8,
    encryption_algorithm = serialization.NoEncryption()
)
with open('private_key.pem', 'wb') as f:
    f.write(pem)
        
#---serialize the public key as bytes---
pem = public_key.public_bytes(
    encoding = serialization.Encoding.PEM,
    format = serialization.PublicFormat.SubjectPublicKeyInfo
)

with open('public_key.pem', 'wb') as f:
    f.write(pem)

In [ ]:
with open('private_key.pem', 'rb') as f:
    private_key = serialization.load_pem_private_key(
        f.read(),
        password = None,
        backend = default_backend()
    )
    
with open('public_key.pem', 'rb') as f:
    public_key = serialization.load_pem_public_key(
        f.read(),
        backend = default_backend()
    )

In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

plaintext = bytes("This message is secret.",'utf-8')

# encrypt the message using the public key
ciphertext = public_key.encrypt(
    plaintext,
    padding.OAEP(
        mgf = padding.MGF1(algorithm = hashes.SHA256()),
        algorithm = hashes.SHA256(),
        label = None
    )
)

In [ ]:
# decrypt using the private key
plaintext = private_key.decrypt(
    ciphertext,
    padding.OAEP(
        mgf = padding.MGF1(algorithm = hashes.SHA256()),
        algorithm = hashes.SHA256(),
        label = None
    )
)
print(plaintext.decode('utf-8'))
# This message is secret.

In [ ]:
# private_key.encrypt(...)    # ERROR
# AttributeError: '_RSAPrivateKey' object has no attribute 'encrypt'

import base64

plaintext = bytes("This message is public.",'utf-8')

# sign the message using the private key
signed = private_key.sign(
    plaintext,
    padding.PSS(
        mgf = padding.MGF1(algorithm=hashes.SHA256()),
        salt_length = padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

# print out the digital signature using base64 encoding
signed_base64 = base64.b64encode(signed).decode('utf-8')
print(signed_base64)

In [ ]:
from cryptography.exceptions import InvalidSignature

# decode the digital signature from base64
signed = base64.b64decode(signed_base64)

try:
    public_key.verify(
        signed,
        plaintext,    # from the previous section
        padding.PSS(
            mgf = padding.MGF1(hashes.SHA256()),
            salt_length = padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    print('Signature is valid!')
except InvalidSignature:
    print('Signature is invalid!')